In [71]:
import pandas as pd
import requests
import time

In [62]:
users = pd.read_excel("data/users_enriched.xlsx")
users_filtered = users.loc[users['final_decision'] == 1]
users_filtered

,user_id,source,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,...,created_at,updated_at,is_student,is_employee,is_currently_employed,is_research_group,final_decision,note,new_entry,date
3,73616e646572,employees_github_usernames.csv,6605233.0,MDQ6VXNlcjY2MDUyMzM=,https://avatars.githubusercontent.com/u/660523...,NaN,https://api.github.com/users/73616e646572,https://github.com/73616e646572,https://api.github.com/users/73616e646572/foll...,https://api.github.com/users/73616e646572/foll...,...,2014-02-06T13:51:56Z,2021-11-19T12:23:00Z,False,1.0,1.0,0.0,1,0,False,2021-12-02
4,a-quei,ids_search_users.csv,6829836.0,MDQ6VXNlcjY4Mjk4MzY=,https://avatars.githubusercontent.com/u/682983...,NaN,https://api.github.com/users/a-quei,https://github.com/a-quei,https://api.github.com/users/a-quei/followers,https://api.github.com/users/a-quei/following{...,...,2014-03-02T08:21:10Z,2021-08-12T20:33:30Z,False,0.0,0.0,0.0,1,0,False,2021-12-02
9,ajueling,ids_search_repos.csv,20947797.0,MDQ6VXNlcjIwOTQ3Nzk3,https://avatars.githubusercontent.com/u/209477...,NaN,https://api.github.com/users/AJueling,https://github.com/AJueling,https://api.github.com/users/AJueling/followers,https://api.github.com/users/AJueling/followin...,...,2016-08-10T11:21:34Z,2021-11-11T14:52:36Z,False,1.0,1.0,0.0,1,0,False,2021-12-02
10,albertgatt,employees_github_usernames.csv,8494566.0,MDQ6VXNlcjg0OTQ1NjY=,https://avatars.githubusercontent.com/u/849456...,NaN,https://api.github.com/users/albertgatt,https://github.com/albertgatt,https://api.github.com/users/albertgatt/followers,https://api.github.com/users/albertgatt/follow...,...,2014-08-19T17:30:50Z,2021-11-03T15:58:02Z,False,1.0,1.0,0.0,1,NaN,True,2021-12-02
12,aldertzomer,ids_search_users.csv,10392411.0,MDQ6VXNlcjEwMzkyNDEx,https://avatars.githubusercontent.com/u/103924...,NaN,https://api.github.com/users/aldertzomer,https://github.com/aldertzomer,https://api.github.com/users/aldertzomer/follo...,https://api.github.com/users/aldertzomer/follo...,...,2015-01-04T19:49:33Z,2021-11-17T18:27:58Z,False,0.0,0.0,0.0,1,0,False,2021-12-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,wouter-swierstra,ids_search_repos.csv,1588245.0,MDQ6VXNlcjE1ODgyNDU=,https://avatars.githubusercontent.com/u/158824...,NaN,https://api.github.com/users/wouter-swierstra,https://github.com/wouter-swierstra,https://api.github.com/users/wouter-swierstra/...,https://api.github.com/users/wouter-swierstra/...,...,2012-03-29T19:25:32Z,2021-10-22T14:30:05Z,False,1.0,1.0,0.0,1,0,False,2021-12-02
478,wouterdroogers,ids_search_users.csv,38694532.0,MDQ6VXNlcjM4Njk0NTMy,https://avatars.githubusercontent.com/u/386945...,NaN,https://api.github.com/users/WouterDroogers,https://github.com/WouterDroogers,https://api.github.com/users/WouterDroogers/fo...,https://api.github.com/users/WouterDroogers/fo...,...,2018-04-24T13:44:53Z,2021-01-07T15:10:19Z,False,1.0,1.0,0.0,1,0,False,2021-12-02
486,yuntaoj,ids_search_users.csv,74980142.0,MDQ6VXNlcjc0OTgwMTQy,https://avatars.githubusercontent.com/u/749801...,NaN,https://api.github.com/users/yuntaoj,https://github.com/yuntaoj,https://api.github.com/users/yuntaoj/followers,https://api.github.com/users/yuntaoj/following...,...,2020-11-24T16:38:30Z,2021-03-08T10:03:10Z,False,0.0,0.0,0.0,1,0,False,2021-12-02
487,zievathustra,ids_search_repos.csv,61464979.0,MDQ6VXNlcjYxNDY0OTc5,https://avatars.githubusercontent.com/u/614649...,NaN,https://api.github.com/users/zievathustra,https://github.com/zievathustra,https://api.github.com/users/zievathustra/foll...,https://api.github.com/users/zievathustra/foll...,...,2020-02-25T14:22:18Z,2021-11-09T10:56:39Z,False,1.0,1.0,0.0,1,0,False,2021-12-02


# Faculties
* Faculty of Geosciences
* Faculty of Humanities
* Faculty of Law, Economics and Governance
* Faculty of Medicine
* Faculty of Science
* Faculty of Social and Behavioural Sciences
* Faculty of Veterinary Medicine

In [82]:

name = "Gerko Vink"
def search_employee_url(name):
    if "nan" == name:
        return None
    name_url = name.replace(" ", "%20")
    url = f"https://www.uu.nl/medewerkers/RestApi/Public/search?expression=%7B%22q%22%3A%22{name_url}%22%2C%22mode%22%3A%22naam%22%7D&t=null&l=EN"
    json_nested  = requests.get(url)
    try:
        employee_url = json_nested.json()["Employees"][0]["Url"]
#         print(name, employee_url)
        return employee_url
    except Exception as e:
        return None
    
search_employee_url(name)

'GVink'

In [85]:
employee_url = "GVink"
def get_employee_info(employee_url):
    if employee_url is None:
        return [None]*3
    API_link = requests.get(f"https://www.uu.nl/medewerkers/RestApi/Public/getEmployeeData?page={employee_url}&l=EN")
    API_json = API_link.json()["Employee"]
    # loop since faculties could be more than one?
    fac = [[faculty["Faculty"], faculty["FacultyUrl"]] for faculty in API_json["Faculties"]] 
    positions = [API_json["Positions"][0]["Level1"], 
                 API_json["Positions"][0]["Level2"], 
                 API_json["Positions"][0]["Level3"], 
                 API_json["Positions"][0]["Level4"], 
                 API_json["Positions"][0]["Position"]]
    info = [API_json["NameShort"], fac, positions]
    return info

get_employee_info(employee_url)

['Gerko Vink',
 [['Social and Behavioural Sciences',
   'https://www.uu.nl/en/organisation/faculty-of-social-and-behavioural-sciences']],
 ['Social and Behavioural Sciences',
  'Social Sciences',
  'Methodology and Statistics',
  None,
  'Associate Professor']]

In [87]:
employee_urls = []
for user in users_filtered["name"]:
    result = search_employee_url(str(user))
    print(result)
    employee_urls.append(result)
#     time.sleep(1)
print(employee_urls)

In [90]:
employee_infos = []
for employee_url in employee_urls:
    result = get_employee_info(employee_url)
    print(result)
    employee_infos.append(result)

['Casper Arends', [['Humanities', 'https://www.uu.nl/en/organisation/faculty-of-humanities']], ['Humanities', 'Student Affairs, Education and Research', 'General', None, 'Student Affairs Assistant']]
['G. (Guanyi) Chen', [['Science', 'https://www.uu.nl/en/organisation/faculty-of-science']], ['Science', 'Information and computing sciences', 'Intelligent Software Systems', 'Natural Language Processing', 'Lecturer']]
['A. (Andre) Jüling', [['UU', None]], ['UU', '', '', '', None]]
['A. (Albert) Gatt', [['Science', 'https://www.uu.nl/en/organisation/faculty-of-science']], ['Science', 'Information and computing sciences', 'Intelligent Software Systems', 'Natural Language Processing', 'Associate Professor']]
['A.L. (Aldert) Zomer', [['Veterinary Medicine', 'https://www.uu.nl/en/organisation/faculty-of-veterinary-medicine']], ['Veterinary Medicine', 'Department Biomolecular Health Sciences', 'Infectious Diseases & Immunology', ' Clinical Infectiology', 'Assistant Professor']]
[None, None, None

['Jaap Nienhuis', [['Geosciences', 'https://www.uu.nl/en/organisation/faculty-of-geosciences']], ['Geosciences', 'Physical Geography', 'Coastal Dynamics, Fluvial Systems and Global Change', 'Global Change Geomorphology', 'Assistant Professor']]
['Johannes Bijlsma', [['Law, Economics and Governance', 'https://www.uu.nl/en/organisation/faculty-of-law-economics-and-governance']], ['Law, Economics and Governance', 'Utrecht University School of Law', 'Willem Pompe Institute for Criminal Law and Criminology', None, 'Associate Professor']]
[None, None, None]
[None, None, None]
['Joost Gadellaa', [['UU', None]], ['UU', '', '', '', None]]
['Jan-Willem Simons', [['Social and Behavioural Sciences', 'https://www.uu.nl/en/organisation/faculty-of-social-and-behavioural-sciences']], ['Social and Behavioural Sciences', 'Social Sciences', 'Sociology', None, 'PhD Candidate']]
[None, None, None]
[None, None, None]
['Konstantinos Kogkalidis', [['Humanities', 'https://www.uu.nl/en/organisation/faculty-of-h